# This notebook contains the code for section 3.4


In [2]:
# using Pkg;
# Pkg.add("DynamicPolynomials")
# Pkg.add("MosekTools")
# Pkg.add("Symbolics")

using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using Symbolics

# Radiated Gravitational Wave Power Equation

In [ ]:
@show 1+1

@polyvar P x y Ω r G m₁ m₂ M a b d e f c Q₁₁ Q₁₂ Q₂₂ 

vars =[P x y Ω G r c m₁ m₂ Q₁₁ Q₁₂ Q₂₂]

function differentiateTrig2(p) # p is a bivariate degree-2 trigonometric polynomials in x,y with variable coefficients
    
    q=2*Ω*y*div(sum(monomials(p).*coefficients(p).*(DynamicPolynomials.degree.(monomials(p), x).==2)), x)
    q+=-2*Ω*x*div(sum(monomials(p).*coefficients(p).*(DynamicPolynomials.degree.(monomials(p), y).==2)), y)
    q+=Ω*(-x^2+y^2)*div(sum(monomials(p).*coefficients(p).*(DynamicPolynomials.degree.(monomials(p), x).==1).*(DynamicPolynomials.degree.(monomials(p), y).==1)), x*y)
    q+=Ω*y*div(sum(monomials(p).*coefficients(p).*(DynamicPolynomials.degree.(monomials(p), x).==1).*(DynamicPolynomials.degree.(monomials(p), y).==0)), x)
    q+=Ω*x*div(sum(monomials(p).*coefficients(p).*(DynamicPolynomials.degree.(monomials(p), x).==0).*(DynamicPolynomials.degree.(monomials(p), y).==1)), y)

    return q
end

function tripDiffTrig2(p)
   return  differentiateTrig2(differentiateTrig2(differentiateTrig2(p)))
end


#Q=m₁*m₂*r^2*[[y^2-1/3 x*y 0]; [x*y x^2-1/3 0]; [0 0 -1/3]]
# Q=m₁*m₂*r^2*[[y^2-1/3 x*y 0]; [x*y x^2-1/3 0]; [0 0 -1/3]]

# Let us now define our axioms
axioms=[
    x^2+y^2-1, # Definition of sine and cosine
    Ω^2*r^3-G*(m₁+m₂), #Kepler's third law w/out G
    (m₁+m₂)*Q₁₁-m₁*m₂*r^2*tripDiffTrig2(y^2-1/3),
    (m₁+m₂)*Q₁₂-m₁*m₂*r^2*tripDiffTrig2(x*y),
    (m₁+m₂)*Q₂₂-m₁*m₂*r^2*tripDiffTrig2(x^2-1/3),
    5*c^5*P+G*tr([Q₁₁ Q₁₂; Q₁₂ Q₂₂]^2)
    ]


    function all_monomials_up_to_max_deg_overall(x, deg, deg_overall, deg_elem)
        if size(x,1) == 0
            [1]
        else
        [ x[1]^k * m for k=0:min(deg, deg_overall, deg_elem[1]) 
                    for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k, deg_elem[2:end])
        ]
        end
    end
    
    function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, deg_elem, u)
        [m
            for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall, deg_elem)
            #if all(unit(m) .== u)
        ]
    end
                    
    function degree_poly(p)
        maximum(DynamicPolynomials.degree.(monomials(p)))
    end

    deg = 6
    deg_overall=20
    deg_overall_q=18
    
    deg_elementwise=[1, 4, 4, 4, 3, 6, 1, 5, 5, 2, 2, 2]     #vars =[P x y Ω G r c m₁ m₂ Q_11 Q_12 Q_22
    deg_elementwise_q=[1, 4, 5, 5, 5, 5] #vars =[P G r c m₁ m₂] #deg(r)=5, deg(c)=5, deg(G)=4
    
    candidate_mons = [
        mons_of_max_degree_and_unit_overall(vars, deg, deg_overall, deg_elementwise, [])
        for ai=axioms
    ]
    @show size.(candidate_mons)
    
    model = Model(Mosek.Optimizer)
    mons_q = mons_of_max_degree_and_unit_overall([P, G, r, c, m₁, m₂] , deg, deg_overall_q, deg_elementwise_q, []) # c, 
    coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")
    
    q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
    coeff_αs = [
        @variable(model, [1:size(X,1)], base_name="α$i")
        for (i,X)=enumerate(candidate_mons)
        ]
    @show size.(coeff_αs)
    αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]
    
    residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
    eqs = coefficients(residual)
    
    # Ensure that the sum of the coefficients on the terms involving P isn't zero, in order that P is part of the final expression
    @constraint model sum(coeff_q[DynamicPolynomials.degree.(mons_q, P).>0]) == 1.0
    
    @constraint model eqs .== 0
    @objective model Max 0

    optimize!(model)
    @show termination_status(model)

    value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

    value_q = value_poly(q)
    @show value_q

    @show value_αs = value_poly.(αs)

### Output from the MIT SuperCloud environment
```
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
1 + 1 = 2
tr(tripDiffTrig2.(Q) ^ 2) = 32.0x⁴Ω⁶r⁴m₁²m₂² + 64.0x²y²Ω⁶r⁴m₁²m₂² + 32.0y⁴Ω⁶r⁴m₁²m₂²
size.(candidate_mons) = [(416392,), (416392,), (416392,)]
size.(coeff_αs) = [(416392,), (416392,), (416392,)]
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 2847899         
  Cones                  : 0               
  Scalar variables       : 1261433         
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.97            
Lin. dep.  - number                 : 52533           
Presolve terminated. Time: 2.60    
Optimizer terminated. Time: 3.98    

termination_status(model) = MathOptInterface.OPTIMAL
value_q = 0.25Pr⁵m₁²c⁵ + 0.5Pr⁵m₁m₂c⁵ + 0.25Pr⁵m₂²c⁵ + 1.6G⁴m₁⁵m₂² + 4.800000000000001G⁴m₁⁴m₂³ + 4.800000000000001G⁴m₁³m₂⁴ + 1.6G⁴m₁²m₂⁵
value_αs = value_poly.(αs) = Polynomial{true, Float64}[-1.6x²Ω⁴r⁶G²m₁³m₂² - 1.6x²Ω⁴r⁶G²m₁²m₂³ - 1.6y²Ω⁴r⁶G²m₁³m₂² - 1.6y²Ω⁴r⁶G²m₁²m₂³ - 1.6Ω⁴r⁶G²m₁³m₂² - 1.6Ω⁴r⁶G²m₁²m₂³, -1.6x⁴Ω⁴r⁶Gm₁²m₂² - 3.2x²y²Ω⁴r⁶Gm₁²m₂² - 1.6y⁴Ω⁴r⁶Gm₁²m₂² - 1.6Ω²r³G²m₁³m₂² - 1.6Ω²r³G²m₁²m₂³ - 1.6G³m₁⁴m₂² - 3.2G³m₁³m₂³ - 1.6G³m₁²m₂⁴, 0.05r⁵]
```